In [17]:
import pandas as pd
import os

In [18]:
names = ['user_id', 'movie_id', 'rating', 'timestamp']
train1 = pd.read_csv(os.path.join('ml-100k', 'u3.base'), '\t', names=names,engine='python')
train1.drop(['timestamp'], axis = 1)

,user_id,movie_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,6,5
...,...,...,...
79995,943,1067,2
79996,943,1074,4
79997,943,1188,3
79998,943,1228,3


In [19]:
test1 = pd.read_csv(os.path.join('ml-100k', 'u3.test'), '\t', names=names,engine='python')
test1.drop(['timestamp'], axis = 1)

,user_id,movie_id,rating
0,1,5,3
1,1,11,2
2,1,16,5
3,1,25,4
4,1,35,1
...,...,...,...
19995,872,332,3
19996,875,169,5
19997,875,772,5
19998,876,294,4


In [20]:
comp_data = pd.read_csv(os.path.join('ml-100k', 'u3.base'), '\t', names=names,engine='python')
user_data = comp_data.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)
user_data.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,0.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# User Similarity Matrix using Cosine similarity as a similarity measure between Users
user_similarity = cosine_similarity(user_data)
user_similarity[np.isnan(user_similarity)] = 0
print(user_similarity)
print(user_similarity.shape)

[[1.         0.11014547 0.04761542 ... 0.16207001 0.16702224 0.38613487]
 [0.11014547 1.         0.11582564 ... 0.15069285 0.16638408 0.05346462]
 [0.04761542 0.11582564 1.         ... 0.1057732  0.13938657 0.02774425]
 ...
 [0.16207001 0.15069285 0.1057732  ... 1.         0.1016418  0.09511958]
 [0.16702224 0.16638408 0.13938657 ... 0.1016418  1.         0.18246466]
 [0.38613487 0.05346462 0.02774425 ... 0.09511958 0.18246466 1.        ]]
(943, 943)


In [22]:
neighbours = user_similarity[user_similarity[:,1] > 0.5]

In [23]:
user_similarity[user_similarity< 0.5] = 0.0
#ini_array1[ini_array1<0] = 0

In [24]:
print(user_similarity)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [25]:
print(user_similarity.shape)


(943, 943)


In [26]:
train1_piv = train1.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)
test1_piv = test1.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)

In [27]:
user_predicted_ratings = np.dot(user_similarity, user_data)
user_predicted_ratings

array([[7.1152664 , 4.5864498 , 5.5864498 , ..., 0.        , 0.        ,
        1.5864498 ],
       [4.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [5.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.54833264, 6.51513031, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [28]:
user_predicted_ratings.shape

(943, 1650)

In [29]:
dummy_train = train1.copy()
dummy_test = test1.copy()

dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x > 0 else 0)

In [30]:
user_predicted_ratings.shape

(943, 1650)

In [31]:
# The movies not rated by user is marked as 1 for prediction 
dummy_train = dummy_train.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(1)

# The movies not rated by user is marked as 0 for evaluation 
dummy_test = dummy_test.pivot(index ='user_id', columns = 'movie_id', values = 'rating').fillna(0)

In [32]:
user_final_ratings = np.multiply(user_predicted_ratings, dummy_train)
user_final_ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,1.58645,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.58645
2,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
3,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
4,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
5,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


In [33]:
test_user_features = test1.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)
test_user_similarity = cosine_similarity(test_user_features)
test_user_similarity[np.isnan(test_user_similarity)] = 0

print(test_user_similarity)
print("- "*10)
print(test_user_similarity.shape)

[[1.         0.07227819 0.         ... 0.14052486 0.         0.        ]
 [0.07227819 1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.14052486 0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
- - - - - - - - - - 
(869, 869)


In [34]:
test_user_similarity[test_user_similarity< 0.5] = 0.0

In [35]:
user_predicted_ratings_test = np.dot(test_user_similarity, test_user_features)
user_predicted_ratings_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [36]:
test_user_final_rating = np.multiply(user_predicted_ratings_test, dummy_test)
test_user_final_rating.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1654,1655,1656,1657,1658,1660,1661,1662,1664,1669
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
train1['rating'].describe()

count    80000.000000
mean         3.531087
std          1.129170
min          1.000000
25%          3.000000
50%          4.000000
75%          4.000000
max          5.000000
Name: rating, dtype: float64

In [38]:
from sklearn.preprocessing import MinMaxScaler

X = test_user_final_rating.copy() 
X = X[X > 0] # only consider non-zero values as 0 means the user haven't rated the movies

scaler = MinMaxScaler(feature_range = (0.5, 5))
scaler.fit(X)
pred = scaler.transform(X)

print(pred)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [39]:
total_non_nan = np.count_nonzero(~np.isnan(pred))
total_non_nan

20000

In [40]:
test = test1.pivot(index = 'user_id', columns = 'movie_id', values = 'rating')
test.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1654,1655,1656,1657,1658,1660,1661,1662,1664,1669
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# Mean abslute error

mae = np.abs(pred - test).sum().sum()/total_non_nan
print(mae)

0.6540696016286441
